<a href="https://colab.research.google.com/github/FarahAhmedAtef/Computer-Vision-based-Autonomous-Robotic-Pick-and-Place-System/blob/main/Flask_API_Server_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, jsonify, request
import subprocess
import json

app = Flask(__name__)

@app.route('/detect', methods=['GET','POST'])
def detect_objects():

    try:
        process = subprocess.Popen(
            ["python", "run_model.py", "--model=my_modellast_ncnn_model", "--source=picamera0", "--resolution=1640x1232"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        # Read stdout line-by-line to avoid buffering issues
        for line in process.stdout:
            line = line.strip()
            try:
                result_data = json.loads(line)
                process.terminate()  # stop the process early if needed
                return jsonify(result_data)  # return as soon as JSON is found
            except json.JSONDecodeError:
                continue  # not JSON, skip

        # If no valid JSON was found
        return jsonify({"error": "No JSON found in output"}), 500

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/print', methods=['POST'])
def print_message():
    data = request.get_json()
    message = data.get("message", "<no message>")
    print(f"[From NiFi] {message}")
    return jsonify({"status": "printed"}), 200
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, threaded=True)